In [1]:
import pandas as pd
import os
work_path = os.getcwd()
print(work_path)

/Users/fangxinniu/Documents/data sense


In [4]:
path = work_path+'/temps.csv'
data = pd.read_csv(path)
data.head(5)

,year,month,day,week,temp_2,temp_1,average,actual,forecast_noaa,forecast_acc,forecast_under,friend
0,2016,1,1,Fri,45,45,45.6,45,43,50,44,29
1,2016,1,2,Sat,44,45,45.7,44,41,50,44,61
2,2016,1,3,Sun,45,44,45.8,41,43,46,47,56
3,2016,1,4,Mon,44,41,45.9,40,44,48,46,53
4,2016,1,5,Tues,41,40,46.0,44,46,46,46,41


# identify anomalies

In [5]:
print('the shape of feature is',data.shape)

the shape of feature is (348, 12)


In [7]:
data.isnull().sum()
data.describe() # to see any data points appear as anomalies and no zeros in any of the measurement columns.
# or make basic plots to spot anomalies

year              0
month             0
day               0
week              0
temp_2            0
temp_1            0
average           0
actual            0
forecast_noaa     0
forecast_acc      0
forecast_under    0
friend            0
dtype: int64

In [8]:
data = pd.get_dummies(data)
data.head()

,year,month,day,temp_2,temp_1,average,actual,forecast_noaa,forecast_acc,forecast_under,friend,week_Fri,week_Mon,week_Sat,week_Sun,week_Thurs,week_Tues,week_Wed
0,2016,1,1,45,45,45.6,45,43,50,44,29,1,0,0,0,0,0,0
1,2016,1,2,44,45,45.7,44,41,50,44,61,0,0,1,0,0,0,0
2,2016,1,3,45,44,45.8,41,43,46,47,56,0,0,0,1,0,0,0
3,2016,1,4,44,41,45.9,40,44,48,46,53,0,1,0,0,0,0,0
4,2016,1,5,41,40,46.0,44,46,46,46,41,0,0,0,0,0,1,0


# train - test
## 这里数据类型变换重点看一下

In [11]:
import numpy as np
# labels is the value we want to predict
labels = np.array(data['actual'])
#remove the labels from data
data = data.drop('actual',axis = 1)
# saving feature name for later use
feature_list = list(data.columns)
data = np.array(data)

In [13]:
from sklearn.model_selection import train_test_split
train_features, test_features, train_labels, test_labels = train_test_split(data, labels, test_size = 0.25, random_state = 42)
print('Training Features Shape:', train_features.shape)
print('Training Labels Shape:', train_labels.shape)
print('Testing Features Shape:', test_features.shape)
print('Testing Labels Shape:', test_labels.shape)


Training Features Shape: (261, 17)
Training Labels Shape: (261,)
Testing Features Shape: (87, 17)
Testing Labels Shape: (87,)


 ## establish the baseline

Before we can make and evaluate predictions, we need to establish a baseline, a sensible measure that we hope to beat with our model. If our model cannot improve upon the baseline, then it will be a failure and we should try a different model or admit that machine learning is not right for our problem.

In [16]:
baseline_preds = test_features[:, feature_list.index('average')] # test组的avg值
baseline_errors = abs(baseline_preds - test_labels) # avg-实际值
print('avg baseline error:',round(np.mean(baseline_errors),2))

avg baseline error: 5.06


# train model (basic)

In [18]:
from sklearn.ensemble import RandomForestRegressor
rf = RandomForestRegressor(n_estimators = 1000, random_state = 42)
rf.fit(train_features, train_labels)

RandomForestRegressor(n_estimators=1000, random_state=42)

In [20]:
prediction = rf.predict(test_features)
error = abs(prediction-test_labels)
print('mean absolute error:',round(np.mean(error),2))

mean absolute error: 3.87


In [21]:
# calculate mean absolute percentage error (MAPE)
mape = 100 * (error/test_labels)
accuracy = 100 - np.mean(mape)
print('accuracy',round(accuracy,2),'%')

accuracy 93.93 %


# improve model (hyperparameter runing)

# interpret model and report result

At this point, we know our model is good, but it’s pretty much a black box. We feed in some Numpy arrays for training, ask it to make a prediction, evaluate the predictions, and see that they are reasonable. The question is: how does this model arrive at the values? There are two approaches to get under the hood of the random forest: first, we can look at a single tree in the forest, and second, we can look at the feature importances of our explanatory variables.

### visualizing a single decision tree

In [23]:
from sklearn.tree import export_graphviz
import pydot

# pull out one tree from the forest
tree = rf.estimators_[8]
export_graphviz(tree, out_file = 'tree.dot', feature_names = feature_list, rounded = True, precision = 1)
(graph, ) = pydot.graph_from_dot_file('tree.dot')
graph.write_png('tree.png')

ModuleNotFoundError: No module named 'pydot'

### variable importance

In [24]:
importances = list(rf.feature_importances_)
importances

[0.0,
 0.010329714202200255,
 0.021119330680396488,
 0.02099328360465877,
 0.655553271473427,
 0.15032982921113514,
 0.045382293248822446,
 0.03485937239968542,
 0.023190478556827192,
 0.020684669325819506,
 0.003524921225987202,
 0.0025881281964305588,
 0.0036125047603753953,
 0.0022890741053675223,
 0.001266445386981603,
 0.002302658049751997,
 0.00197402557213355]

In [32]:
# list of tuples with variables and importance
# rf.feature_importances_和feature_list的顺序是一样的
feature_importances = [(feature, round(importance,2)) for feature, importance in zip(feature_list, importances)]
# sort the variables by most impt first
feature_importances = sorted(feature_importances, key = lambda x:x[1], reverse = True)

In [33]:
[print('variable: {:20} importance: {}'.format(*pair)) for pair in feature_importances]

variable: temp_1               importance: 0.66
variable: average              importance: 0.15
variable: forecast_noaa        importance: 0.05
variable: forecast_acc         importance: 0.03
variable: day                  importance: 0.02
variable: temp_2               importance: 0.02
variable: forecast_under       importance: 0.02
variable: friend               importance: 0.02
variable: month                importance: 0.01
variable: year                 importance: 0.0
variable: week_Fri             importance: 0.0
variable: week_Mon             importance: 0.0
variable: week_Sat             importance: 0.0
variable: week_Sun             importance: 0.0
variable: week_Thurs           importance: 0.0
variable: week_Tues            importance: 0.0
variable: week_Wed             importance: 0.0


[None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None]

In [34]:
rf.feature_importances_

array([0.        , 0.01032971, 0.02111933, 0.02099328, 0.65555327,
       0.15032983, 0.04538229, 0.03485937, 0.02319048, 0.02068467,
       0.00352492, 0.00258813, 0.0036125 , 0.00228907, 0.00126645,
       0.00230266, 0.00197403])

In future implementations of the model, we can remove those variables that have no importance and the performance will not suffer. Additionally, if we are using a different model, say a support vector machine, we could use the random forest feature importances as a kind of feature selection method. Let’s quickly make a random forest with only the two most important variables, the max temperature 1 day prior and the historical average and see how the performance compares.

if we only use the most impt two variables, let's see how performance compares.

In [35]:
impt_index = [feature_list.index('temp_1'),feature_list.index('average')]
train_impt = train_features[:,impt_index]
test_impt = test_features[:,impt_index]


In [36]:
rf_impt = RandomForestRegressor(n_estimators = 1000, random_state = 42)
rf_impt.fit(train_impt, train_labels)
impt_prediction = rf_impt.predict(test_impt)
impt_error = abs(impt_prediction-test_labels)
print('mean absolute error:',round(np.mean(impt_error),2))

mean absolute error: 3.92


3.92 >3.86 it told us we do not need all the data we collected to make accurate predictions.

In [ ]:
take away
1. establish the baseline
2. 


In [7]:
import random 
cards = ['empty', 'empty', 'amazon'] 
random.shuffle(cards) 
a = input('Please choose a card: 0, 1 or 2\n>') 
number = int(a) 
if cards[number] == 'amazon':
    print('you guessed right!') 
else: 
    print('nope, nothing there') 

KeyboardInterrupt: Interrupted by user